# Part 1: Rainfall Data Engineering

# Step 1: Initial Structure Inspection

**Objective:**
Before processing the entire batch of 149 files, we must understand the structure of the data by inspecting a single sample file. This step prevents blind merging errors caused by mismatched headers or data type inconsistencies.

**Methodology:**

1. **Locate Files:** Use the `glob` library to find all `.xlsx` files in the source directory.
2. **Load Sample:** Read the first available file into a pandas DataFrame.
3. **Inspect Schema:**
   - **Column Headers:** Check naming conventions and spacing (e.g., `'Station Name'`).
   - **Data Types:** Verify that numeric fields like `Year` and `Rainfall (mm)` are not stored as text.
   - **Data Preview:** Display the first 5 rows to understand structure and formatting.

In [1]:
import pandas as pd
import glob
import os

# Path configuration
folder_path = r"rain_fall"

# Get a list of all Excel files
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

if all_files:
    # Read the first file found
    first_file = all_files[0]
    df_sample = pd.read_excel(first_file)

    print(f"--- Inspecting: {os.path.basename(first_file)} ---")
    print("\n1. Column Headers:")
    print(df_sample.columns.tolist())

    print("\n2. First 5 rows of data:")
    print(df_sample.head())

    print("\n3. Data Types:")
    print(df_sample.dtypes)
else:
    print("No Excel files found in the specified directory.")

--- Inspecting: Monthly Rainfall in (2014)_01.xlsx ---

1. Column Headers:
['Station Name', 'State', 'District', 'Month', 'Year', 'Rainfall (mm)']

2. First 5 rows of data:
                 Station Name       State         District Month    Year  \
0                      PILANI   Rajasthan           PILANI   Jan  2014.0   
1          MANGALORE BAJPE(A)   Karnataka  DAKSHIN KANNADA   Jan  2014.0   
2                    JHALAWAR   Rajasthan         JHALAWAR   Jan  2014.0   
3                      TEZPUR       Assam         SONITPUR   Jan  2014.0   
4  COIMBATORE / PEELAMEDU (A)  Tamil Nadu       COIMBATORE   Jan  2014.0   

   Rainfall (mm)  
0            0.0  
1            0.0  
2           29.0  
3            0.3  
4            0.0  

3. Data Types:
Station Name         str
State                str
District             str
Month                str
Year             float64
Rainfall (mm)    float64
dtype: object


# Step 2: Batch Loading and Data Integrity Audit

**Objective:**
Load all rainfall files (2010–2022) to detect inconsistencies, formatting errors, and unwanted metadata rows.

**Methodology:**

1. **Bulk Read:** Loop through all Excel files and store them in a list.
2. **Concatenate:** Merge all files into a temporary DataFrame (`raw_df`).
3. **Unique Value Inspection:**
   - **States:** Detect spelling inconsistencies or trailing spaces.
   - **Years:** Check for float representations like `2010.0`.
   - **Months:** Validate month naming consistency.
   - **Districts:** Verify duplicates or formatting issues.
   - **Station Names:** Detect footer text such as "Copyright" or URLs.

**Key Insight:**
Footer metadata and empty rows often get imported as data and must be removed before analysis.


In [2]:
folder_path = r"rain_fall"
all_files = glob.glob(os.path.join(folder_path, "*.xlsx"))

# Create an empty list to store dataframes
df_list = []

print(f"Found {len(all_files)} files. Reading them now...")

for filename in all_files:
    try:
        df = pd.read_excel(filename)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {filename}: {e}")

# Combine into a temporary raw dataframe
if df_list:
    raw_df = pd.concat(df_list, ignore_index=True)

    print("\n--- Unique Value Inspection ---")
    print(f"Total Rows Loaded: {len(raw_df)}")

    # 1. Check States
    print(f"\nUnique States ({raw_df['State'].nunique()}):")
    print(raw_df['State'].unique())

    # 2. Check Years
    print(f"\nUnique Years:")
    print(sorted(raw_df['Year'].unique()))

    # 3. Check Months
    print(f"\nUnique Months:")
    print(raw_df['Month'].unique())

    # 4. Check Districts (Added as requested)
    print(f"\nUnique Districts ({raw_df['District'].nunique()}):")
    print(raw_df['District'].unique())

    # 5. Check Station Names
    print(f"\nUnique Station Names ({raw_df['Station Name'].nunique()}):")
    # Note: This list might be very long
    print(raw_df['Station Name'].unique())

else:
    print("No data loaded.")

Found 149 files. Reading them now...

--- Unique Value Inspection ---
Total Rows Loaded: 36159

Unique States (34):
<StringArray>
[                  'Rajasthan',                   'Karnataka',
                       'Assam',                  'Tamil Nadu',
                     'Gujarat',                 'West Bengal',
                      'Kerala',               'Uttar Pradesh',
                 'Lakshadweep',                      'Punjab',
                 'Uttarakhand',              'Andhra Pradesh',
                         'Goa',            'Himachal Pradesh',
                      'Odisha',           'Jammu and Kashmir',
              'Madhya Pradesh',                     'Haryana',
                 'Maharashtra',                'Chhattisgarh',
                       'Delhi',                       'Bihar',
                   'Meghalaya', 'Andaman and Nicobar Islands',
                   'Telangana',                  'Puducherry',
                     'Tripura',           'Arunacha

# Step 3: Data Cleaning, Formatting, and Final Export

**Objective:**
Transform raw rainfall data into a clean master dataset suitable for analysis and storage.

**Cleaning Logic Applied:**

### 1. Garbage Removal
- Convert `Year` to numeric using coercion (`errors='coerce'`).
- Drop rows missing critical fields:
  - `Year`
  - `Month`
  - `Station Name`

### 2. Type Conversion
- Convert `Year` from float to integer.
- Convert `Rainfall (mm)` to numeric.

### 3. String Sanitization
- Remove leading and trailing whitespace from:
  - `State`
  - `District`
  - `Station Name`
  - `Month`

### 4. Directory Handling
- Create the `results` folder if it does not exist before saving output.

**Output:**
`Final_Rainfall_Data_2010_2022.xlsx`


In [3]:

# Check if raw_df exists from the previous step
if 'raw_df' not in locals():
    print("'raw_df' not found. Please run Code 2 again to load the files first.")
else:
    print("--- 1. STARTING CLEANING PROCESS ---")

    # Create a copy
    df_clean = raw_df.copy()

    # A. Coerce 'Year' to Numeric
    # This is crucial. It turns any text in the Year column (if any) into NaN
    df_clean['Year'] = pd.to_numeric(df_clean['Year'], errors='coerce')

    # B. Remove Empty Rows (The Main Filter)
    # We drop rows where Year, Month, or Station Name is missing (NaN)
    # This automatically removes the "Copyright", "https", and blank rows
    rows_before = len(df_clean)
    df_clean = df_clean.dropna(subset=['Year', 'Month', 'Station Name'])
    rows_after = len(df_clean)

    print(f"Removed {rows_before - rows_after} garbage/footer rows.")

    # C. Fix Formatting
    # Convert Year to integer (2010.0 -> 2010)
    df_clean['Year'] = df_clean['Year'].astype(int)

    # Coerce Rainfall to numeric (turn errors to NaN)
    df_clean['Rainfall (mm)'] = pd.to_numeric(df_clean['Rainfall (mm)'], errors='coerce')

    # Clean text columns (remove extra spaces)
    text_cols = ['Station Name', 'State', 'District', 'Month']
    for col in text_cols:
        df_clean[col] = df_clean[col].astype(str).str.strip()

    # --- 2. VALIDATION ---
    print("\n--- Final Validation ---")
    print(f"Unique Years: {sorted(df_clean['Year'].unique())}")

    # Check if 'Copyright' or 'https' still exists in Station Name
    bad_stations = df_clean[df_clean['Station Name'].str.contains("Copyright|http", case=False)]
    if bad_stations.empty:
        print("Garbage text successfully removed from Station Names.")
    else:
        print(f"Warning: {len(bad_stations)} garbage rows remain.")

# --- 3. SAVE MERGED FILE ---

# 1. Define the subfolder name
output_folder = os.path.join(folder_path, "results")

# 2. Check if it exists. If not, create it!
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Created new folder: {output_folder}")

# 3. Define the full file path
output_file = os.path.join(output_folder, "Final_Rainfall_Data_2010_2022.xlsx")

try:
    df_clean.to_excel(output_file, index=False)
    print(f"\nSUCCESS! Cleaned file saved at:\n{output_file}")
except Exception as e:
    print(f"Error saving file: {e}")

--- 1. STARTING CLEANING PROCESS ---
Removed 596 garbage/footer rows.

--- Final Validation ---
Unique Years: [np.int64(2010), np.int64(2011), np.int64(2012), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022)]
Garbage text successfully removed from Station Names.

SUCCESS! Cleaned file saved at:
rain_fall/results/Final_Rainfall_Data_2010_2022.xlsx


# Step 4: OLTP Database Architecture (Normalization)

**Objective:**
Convert flat Excel storage into a normalized relational database system.

**Normalization Principle (3NF):**
Instead of repeating state and district names thousands of times, store them once and reference them using unique IDs.

**Database Structure:**

### Dimension Tables
- **States** (`StateID`, `StateName`)
- **Districts** (`DistrictID`, `DistrictName`, `StateID`)
- **Stations** (`StationID`, `StationName`, `DistrictID`)

### Fact Table
- **Rainfall_Readings** (`StationID`, `Year`, `Month`, `Rainfall`)

Foreign keys replace textual values to reduce redundancy and improve data integrity.

**Output:**
`Weather_OLTP.db`


In [4]:
import sqlite3
import pandas as pd

# Load the clean data
file_path = r"rain_fall/results/Final_Rainfall_Data_2010_2022.xlsx"
df = pd.read_excel(file_path)

# Connect to SQLite database (creates a file on your disk)
db_path = r"rain_fall\results\Weather_OLTP.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print("--- BUILDING OLTP ARCHITECTURE ---")

# 1. Create Lookup Tables (States, Districts, Stations)
# We use .unique() to get distinct values and assign IDs

# A. States Table
states = df['State'].unique()
df_states = pd.DataFrame(states, columns=['StateName'])
df_states.reset_index(inplace=True)
df_states.rename(columns={'index': 'StateID'}, inplace=True)
df_states.to_sql('States', conn, if_exists='replace', index=False)

# B. Districts Table (Needs StateID)
# Merge original data with State IDs to get the relationship
district_map = df[['State', 'District']].drop_duplicates()
district_map = district_map.merge(df_states, left_on='State', right_on='StateName')
df_districts = district_map[['District', 'StateID']].reset_index()
df_districts.rename(columns={'index': 'DistrictID', 'District': 'DistrictName'}, inplace=True)
df_districts.to_sql('Districts', conn, if_exists='replace', index=False)

# C. Stations Table (Needs DistrictID)
station_map = df[['District', 'Station Name']].drop_duplicates()
station_map = station_map.merge(df_districts, left_on='District', right_on='DistrictName')
df_stations = station_map[['Station Name', 'DistrictID']].reset_index()
df_stations.rename(columns={'index': 'StationID', 'Station Name': 'StationName'}, inplace=True)
df_stations.to_sql('Stations', conn, if_exists='replace', index=False)

# 2. Create the Fact/Transaction Table (Rainfall Readings)
# This replaces names with IDs to save space
fact_df = df.merge(df_stations, left_on='Station Name', right_on='StationName')
final_oltp = fact_df[['StationID', 'Year', 'Month', 'Rainfall (mm)']]
final_oltp.to_sql('Rainfall_Readings', conn, if_exists='replace', index=False)

print("OLTP Database created successfully.")
print("Verifying Table Counts:")
print(f"States: {pd.read_sql('SELECT COUNT(*) FROM States', conn).iloc[0,0]}")
print(f"Districts: {pd.read_sql('SELECT COUNT(*) FROM Districts', conn).iloc[0,0]}")
print(f"Stations: {pd.read_sql('SELECT COUNT(*) FROM Stations', conn).iloc[0,0]}")
print(f"Readings: {pd.read_sql('SELECT COUNT(*) FROM Rainfall_Readings', conn).iloc[0,0]}")

conn.close()

--- BUILDING OLTP ARCHITECTURE ---
OLTP Database created successfully.
Verifying Table Counts:
States: 34
Districts: 329
Stations: 405
Readings: 35563


# Step 5: OLAP Operations (Business Intelligence)

**Objective:**
Move from transactional storage (OLTP) to analytical processing (OLAP) for insight generation.

**Data Enrichment:**
A new dimension called `Season` is derived from `Month`:

- **Winter:** Jan, Feb
- **Summer:** Mar, Apr, May
- **Monsoon:** Jun, Jul, Aug, Sep
- **Post-Monsoon:** Oct, Nov, Dec

**OLAP Operations Implemented:**

1. **Roll-Up:** Aggregate monthly rainfall into yearly totals.
2. **Drill-Down:** Break state-level summaries into district-level insights.
3. **Slice:** Filter data for a single dimension value (e.g., `Year = 2020`).
4. **Dice:** Filter across multiple dimensions (e.g., Kerala during Monsoon).
5. **Pivot:** Cross-tabulate rainfall by `State` and `Year`.


In [5]:
import pandas as pd
import numpy as np

# Load data
file_path = r"rain_fall/results/Final_Rainfall_Data_2010_2022.xlsx"
df = pd.read_excel(file_path)

print("--- OLAP OPERATIONS ---")

# 1. ENRICHMENT (Adding Dimensions)
# Add a 'Season' dimension for better analysis
def get_season(month):
    if month in ['Jun', 'Jul', 'Aug', 'Sep']: return 'Monsoon'
    elif month in ['Oct', 'Nov', 'Dec']: return 'Post-Monsoon'
    elif month in ['Jan', 'Feb']: return 'Winter'
    else: return 'Summer'

df['Season'] = df['Month'].apply(get_season)

# --- OPERATION A: ROLL-UP (Summarize details to a higher level) ---
# Moving from Monthly Data -> Yearly Data
print("\n[A] ROLL-UP: Total Rainfall per Year")
rollup = df.groupby('Year')['Rainfall (mm)'].sum()
print(rollup.head())

# --- OPERATION B: DRILL-DOWN (Break summary into details) ---
# Breaking a specific State (Maharashtra) into Districts
print("\n[B] DRILL-DOWN: Rainfall in Maharashtra (By District)")
drilldown = df[df['State'] == 'Maharashtra'].groupby('District')['Rainfall (mm)'].mean().sort_values(ascending=False)
print(drilldown.head(5))

# --- OPERATION C: SLICE (Filter for one specific dimension) ---
# Taking a single slice of the "Cube": Only the year 2020
print("\n[C] SLICE: Top 5 Wettest States in 2020")
slice_2020 = df[df['Year'] == 2020].groupby('State')['Rainfall (mm)'].mean().sort_values(ascending=False).head(5)
print(slice_2020)

# --- OPERATION D: DICE (Filter on multiple dimensions) ---
# Sub-cube: 'Kerala' AND 'Monsoon' Season
print("\n[D] DICE: Kerala during Monsoon Season")
dice_df = df[(df['State'] == 'Kerala') & (df['Season'] == 'Monsoon')]
print(f"Average Monsoon Rainfall in Kerala: {dice_df['Rainfall (mm)'].mean():.2f} mm")

# --- OPERATION E: PIVOT (The Full OLAP Cube View) ---
# Rows=State, Columns=Year, Values=Rainfall
print("\n[E] PIVOT TABLE (Cross-Tabulation)")
pivot_cube = pd.pivot_table(df, values='Rainfall (mm)', index='State', columns='Year', aggfunc='mean')
print(pivot_cube.iloc[:5, :5]) # Showing first 5 states and first 5 years

--- OLAP OPERATIONS ---

[A] ROLL-UP: Total Rainfall per Year
Year
2010    429768.3
2011    368934.7
2012    379180.4
2013    450832.3
2014    333312.4
Name: Rainfall (mm), dtype: float64

[B] DRILL-DOWN: Rainfall in Maharashtra (By District)
District
RATNAGIRI         276.692523
SATARA            251.237438
GREATER MUMBAI    215.146429
RAIGARH           198.770732
THANE             196.495082
Name: Rainfall (mm), dtype: float64

[C] SLICE: Top 5 Wettest States in 2020
State
Meghalaya                      448.323077
Chhattisgarh                   364.900000
Arunachal Pradesh              310.707143
Kerala                         241.459441
Andaman and Nicobar Islands    217.996667
Name: Rainfall (mm), dtype: float64

[D] DICE: Kerala during Monsoon Season
Average Monsoon Rainfall in Kerala: 436.40 mm

[E] PIVOT TABLE (Cross-Tabulation)
Year                               2010        2011        2012        2013  \
State                                                                    

# Part 2: Crop Data Engineering

# Step 6: Loading the Complex Crop Report

**Objective:**
Load the crop production dataset for processing.

**Challenge:**
The file has a `.xls` extension but is actually an HTML table saved with an Excel extension.

**Solution:**
Use `pandas.read_html()` instead of standard Excel readers to parse the file correctly.


In [6]:
import pandas as pd
import os

# Path to the file
file_path = r"Crop.xls"

# Check if file exists
if os.path.exists(file_path):
    try:
        # ATTEMPT 1: Read as standard legacy Excel (.xls) using xlrd engine
        print("Attempting to read with 'xlrd' engine...")
        df_crop_sample = pd.read_excel(file_path, header=[0, 1], engine='xlrd')
        print("Success! Loaded with xlrd.")

    except Exception as e:
        print(f"Standard load failed: {e}")
        print("\n--- ATTEMPT 2: Checking if file is actually HTML/XML (Web Export) ---")
        # Sometimes websites export HTML tables named as .xls
        try:
            dfs = pd.read_html(file_path, header=[0, 1])
            if dfs:
                df_crop_sample = dfs[0]
                print("Success! File was actually an HTML table.")
                print(df_crop_sample.head())
        except Exception as html_e:
            print(f"HTML load also failed: {html_e}")

else:
    print(f"File not found at: {file_path}")

Attempting to read with 'xlrd' engine...
Standard load failed: `Import xlrd` failed. Install xlrd >= 2.0.1 for xls Excel support Use pip or conda to install the xlrd package.

--- ATTEMPT 2: Checking if file is actually HTML/XML (Web Export) ---
Success! File was actually an HTML table.
                            State     District         Year          Banana  \
                            State     District         Year      Whole Year   
0                           State     District         Year  Area (Hectare)   
1  1. Andaman and Nicobar Islands  1. Nicobars  2010 - 2011          593.00   
2  1. Andaman and Nicobar Islands  1. Nicobars  2011 - 2012          620.50   
3  1. Andaman and Nicobar Islands  1. Nicobars  2012 - 2013          241.00   
4  1. Andaman and Nicobar Islands  1. Nicobars  2013 - 2014          170.00   

                                                      Coconut  \
          Whole Year.1           Whole Year.2      Whole Year   
0  Production (Tonnes)  Yiel

# Step 7: ETL Pipeline — Cleaning and Reshaping Crop Data

**Objective:**
Transform wide-format crop data into a normalized long-format dataset.

**Transformation Steps:**

### 1. Header Parsing
Extract metric names (`Area`, `Production`, `Yield`) embedded within the first data row.

### 2. Identifier Cleaning
- Remove numbering prefixes from `State` and `District`.
- Convert year ranges into integer years.

### 3. Reshaping (Wide to Long)
Convert crop-specific columns into rows so each record represents:

- `State`
- `District`
- `Year`
- `Crop`
- `Area`
- `Production`
- `Yield`

**Output:**
`Final_Crop_Data_2010_2022.xlsx`

In [7]:
import pandas as pd
import numpy as np
import os
import re

file_path = r"Crop.xls"
output_folder = r"Crop_results"

# NEW: Create output directory if it does not exist
os.makedirs(output_folder, exist_ok=True)

# 1. Load the HTML table
dfs = pd.read_html(file_path, header=[0, 1])
df_raw = dfs[0]

print("--- 1. INITIAL PROCESSING ---")

# 2. Extract Metric Names from the first row of actual data
metrics_row = df_raw.iloc[0].values
print(f"Sample Metrics found: {metrics_row[3:6]}")

# 3. Construct New Column Names
new_columns = ['State', 'District', 'Year']
raw_cols = df_raw.columns.tolist()

for i in range(3, len(raw_cols)):
    crop_name = raw_cols[i][0]
    metric_raw = str(metrics_row[i])
    if 'Area' in metric_raw:
        metric = 'Area'
    elif 'Production' in metric_raw:
        metric = 'Production'
    elif 'Yield' in metric_raw:
        metric = 'Yield'
    else:
        metric = 'Unknown'
    new_columns.append(f"{crop_name}_{metric}")

df_raw.columns = new_columns
df_raw = df_raw.drop(0).reset_index(drop=True)

# 4. Clean State, District, and Year
print("\n--- 2. CLEANING IDENTIFIERS ---")

def clean_name(text):
    if pd.isna(text):
        return text
    return re.sub(r'^\d+\.\s*', '', str(text)).strip()

def clean_year(text):
    if pd.isna(text):
        return np.nan
    try:
        return int(str(text).split('-')[0].strip())
    except:
        return np.nan

df_raw['State'] = df_raw['State'].apply(clean_name)
df_raw['District'] = df_raw['District'].apply(clean_name)
df_raw['Year'] = df_raw['Year'].apply(clean_year)

# 5. RESHAPE: Wide to Long
print("\n--- 3. RESHAPING (UNPIVOTING) ---")

processed_frames = []
crop_cols = [c for c in df_raw.columns if '_' in c]
unique_crops = {c.split('_')[0] for c in crop_cols}

for crop in unique_crops:
    cols_to_keep = [
        'State', 'District', 'Year',
        f"{crop}_Area",
        f"{crop}_Production",
        f"{crop}_Yield"
    ]

    temp_df = df_raw[cols_to_keep].copy()
    temp_df.columns = ['State', 'District', 'Year', 'Area', 'Production', 'Yield']
    temp_df['Crop'] = crop
    processed_frames.append(temp_df)

df_final_crop = pd.concat(processed_frames, ignore_index=True)

# 6. Final Clean: Numeric Conversions & Handling NaNs
cols_numeric = ['Area', 'Production', 'Yield']
for col in cols_numeric:
    df_final_crop[col] = pd.to_numeric(df_final_crop[col], errors='coerce')

df_final_crop = df_final_crop.dropna(subset=cols_numeric, how='all')
df_final_crop[cols_numeric] = df_final_crop[cols_numeric].fillna(0)

print(f"\nSUCCESS! Final Dataset Shape: {df_final_crop.shape}")

# 7. Save
output_file = os.path.join(output_folder, "Final_Crop_Data_2010_2022.xlsx")
df_final_crop.to_excel(output_file, index=False)
print(f"Saved to: {output_file}")


--- 1. INITIAL PROCESSING ---
Sample Metrics found: <StringArray>
['Area (Hectare)', 'Production (Tonnes)', 'Yield (Tonne/Hectare)']
Length: 3, dtype: str

--- 2. CLEANING IDENTIFIERS ---

--- 3. RESHAPING (UNPIVOTING) ---

SUCCESS! Final Dataset Shape: (29578, 7)
Saved to: Crop_results/Final_Crop_Data_2010_2022.xlsx


# Part 3: Data Integration and Merging

# Step 8: Gap Analysis and Manual Mapping

**Objective:**
Identify district mismatches between rainfall and crop datasets.

**Examples of Issues:**
- "SPSR Nellore" vs "Nellore"
- Minor spelling variations
- Case sensitivity differences

**Process:**

1. Convert names to uppercase for uniform comparison.
2. Compare crop districts against valid rainfall districts.
3. Generate:
   - `reference_district_list.txt`
   - `manual_mapping_worksheet.csv`

These files allow manual correction of mismatched district names.


In [8]:
import pandas as pd
import sqlite3
import difflib
import os

# Paths
crop_file = r"Crop_results/Final_Crop_Data_2010_2022.xlsx"
db_path = r"rain_fall\results\Weather_OLTP.db"
output_csv = r"rain_fall\results\manual_mapping_worksheet.csv"
reference_txt = r"rain_fall\results\reference_district_list.txt"

print("--- GENERATING MANUAL MAPPING WORKSHEET ---")

# 1. Load Data
df_crop = pd.read_excel(crop_file)
conn = sqlite3.connect(db_path)
df_db_districts = pd.read_sql("SELECT * FROM Districts", conn)
df_db_states = pd.read_sql("SELECT * FROM States", conn)
conn.close()

# 2. Prepare Lists
df_crop['State_Upper'] = df_crop['State'].str.upper().str.strip()
df_crop['District_Upper'] = df_crop['District'].str.upper().str.strip()

df_valid = df_db_districts.merge(df_db_states, on='StateID')
df_valid['StateName_Upper'] = df_valid['StateName'].str.upper().str.strip()
df_valid['DistrictName_Upper'] = df_valid['DistrictName'].str.upper().str.strip()

# 3. Create Reference List
with open(reference_txt, "w") as f:
    f.write("=== REFERENCE LIST ===\n")
    for state in sorted(df_valid['StateName_Upper'].unique()):
        f.write(f"--- {state} ---\n")
        dists = sorted(df_valid[df_valid['StateName_Upper'] == state]['DistrictName_Upper'].tolist())
        for d in dists: f.write(f"{d}\n")
        f.write("\n")

# 4. Identify Mismatches
unique_crop_locs = df_crop[['State_Upper', 'District_Upper']].drop_duplicates()
mapping_data = []

for index, row in unique_crop_locs.iterrows():
    state = row['State_Upper']
    dist = row['District_Upper']
    valid_options = df_valid[df_valid['StateName_Upper'] == state]['DistrictName_Upper'].tolist()

    if dist in valid_options: continue

    matches = difflib.get_close_matches(dist, valid_options, n=1, cutoff=0.0)
    suggestion = matches[0] if matches else ""

    mapping_data.append({
        'State': state,
        'Crop_District_Original': dist,
        'Auto_Suggestion': suggestion,
        'CORRECT_NAME_FROM_DB': suggestion
    })

# 5. Save the Worksheet
df_map = pd.DataFrame(mapping_data).sort_values(by=['State', 'Crop_District_Original'])
df_map.to_csv(output_csv, index=False)
print(f"Worksheet created at: {output_csv}")

--- GENERATING MANUAL MAPPING WORKSHEET ---
Worksheet created at: rain_fall\results\manual_mapping_worksheet.csv


# Part 4: Final Integration & Engineering

# Step 9: Applying Manual Geographic Mapping

**Objective:**
Resolve mismatched District names between the Agriculture and Climate datasets.

**Methodology:**

Instead of relying on fuzzy matching (which produced low accuracy), we apply a hardcoded dictionary containing 333 manual corrections.

- **Source:** Manual audit of the Mismatch Report
- **Logic:** Maps district name variations such as
  `VISAKHAPATANAM` → `VISAKHAPATNAM`
- **Execution:** A new column `District_Final` is created and used as the joining key.

**Output:**
`Final_Merged_Dataset_Clean.xlsx` (Preliminary Merge)


In [9]:
import pandas as pd
import io
import os

# --- 1. SETUP PATHS ---
folder_path = r"rain_fall/results"
crop_file = os.path.join(folder_path, "Final_Crop_Data_2010_2022.xlsx")
rain_file = os.path.join(folder_path, "Final_Rainfall_Data_2010_2022.xlsx")
output_file = os.path.join(folder_path, "Final_Merged_Dataset_Clean.xlsx")

# --- 2. MANUAL MAPPING DATA (Embedded) ---
csv_content = """State,Crop_District_Original,Auto_Suggestion,CORRECT_NAME_FROM_DB
ANDAMAN AND NICOBAR ISLANDS,NICOBARS,NICOBAR,NICOBAR
ANDAMAN AND NICOBAR ISLANDS,NORTH AND MIDDLE ANDAMAN,NORTH & MIDDLE ANDAMAN,NORTH & MIDDLE ANDAMAN
ANDAMAN AND NICOBAR ISLANDS,SOUTH ANDAMANS,SOUTH ANDAMAN,SOUTH ANDAMAN
ANDHRA PRADESH,ANANTAPUR,ANATAPUR,ANATAPUR
ANDHRA PRADESH,KADAPA,KADDAPA,KADDAPA
ANDHRA PRADESH,SPSR NELLORE,NELLORE,NELLORE
ANDHRA PRADESH,VISAKHAPATANAM,VISAKHAPATNAM,VISAKHAPATNAM
ANDHRA PRADESH,VIZIANAGARAM,ANATAPUR,ANATAPUR
ARUNACHAL PRADESH,ANJAW,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,CHANGLANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,DIBANG VALLEY,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,EAST KAMENG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,KAMLE,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,KRA DAADI,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,KURUNG KUMEY,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,LEPARADA,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,LOHIT,TIRAP,TIRAP
ARUNACHAL PRADESH,LONGDING,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,LOWER DIBANG VALLEY,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,LOWER SIANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,LOWER SUBANSIRI,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,NAMSAI,PAPUM PARE,PAPUM PARE
ARUNACHAL PRADESH,PAKKE KESSANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,SHI YOMI,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,SIANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,TAWANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,UPPER SIANG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,UPPER SUBANSIRI,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,WEST KAMENG,EAST SIANG,EAST SIANG
ARUNACHAL PRADESH,WEST SIANG,EAST SIANG,EAST SIANG
ASSAM,BAKSA,DIMA HASAO,DIMA HASAO
ASSAM,BARPETA,KAMRUP (METRO),KAMRUP (METRO)
ASSAM,BISWANATH,SONITPUR,SONITPUR
ASSAM,BONGAIGAON,NOWGAON,NOWGAON
ASSAM,CHARAIDEO,CACHAR,CACHAR
ASSAM,CHIRANG,DARRANG,DARRANG
ASSAM,DHEMAJI,DHUBRI,DHUBRI
ASSAM,HAILAKANDI,GOALPARA,GOALPARA
ASSAM,HOJAI,JORHAT,JORHAT
ASSAM,KAMRUP,KAMRUP (RURAL),KAMRUP (RURAL)
ASSAM,KAMRUP METRO,KAMRUP (METRO),KAMRUP (METRO)
ASSAM,KARBI ANGLONG,DARRANG,DARRANG
ASSAM,KARIMGANJ,DARRANG,DARRANG
ASSAM,KOKRAJHAR,CACHAR,CACHAR
ASSAM,MAJULI,UDALGURI,UDALGURI
ASSAM,MARIGAON,NOWGAON,NOWGAON
ASSAM,NAGAON,NOWGAON,NOWGAON
ASSAM,NALBARI,UDALGURI,UDALGURI
ASSAM,SIVASAGAR,DIBRUGARH,DIBRUGARH
ASSAM,SOUTH SALMARA MANCACHAR,CACHAR,CACHAR
ASSAM,TINSUKIA,DIBRUGARH,DIBRUGARH
ASSAM,WEST KARBI ANGLONG,DARRANG,DARRANG
BIHAR,ARWAL,SARAN,SARAN
BIHAR,AURANGABAD,DARBHANGA,DARBHANGA
BIHAR,BANKA,PATNA,PATNA
BIHAR,BEGUSARAI,SARAN,SARAN
BIHAR,BHOJPUR,BHAGALPUR,BHAGALPUR
BIHAR,BUXAR,BHAGALPUR,BHAGALPUR
BIHAR,GOPALGANJ,SARAN,SARAN
BIHAR,JAMUI,ARARIA,ARARIA
BIHAR,JEHANABAD,DARBHANGA,DARBHANGA
BIHAR,KAIMUR (BHABUA),DARBHANGA,DARBHANGA
BIHAR,KATIHAR,PATNA,PATNA
BIHAR,KHAGARIA,ARARIA,ARARIA
BIHAR,KISHANGANJ,DARBHANGA,DARBHANGA
BIHAR,LAKHISARAI,SARAN,SARAN
BIHAR,MADHEPURA,PURNEA,PURNEA
BIHAR,MADHUBANI,DARBHANGA,DARBHANGA
BIHAR,MUNGER,MUZAFFARPUR,MUZAFFARPUR
BIHAR,NALANDA,SARAN,SARAN
BIHAR,NAWADA,ARARIA,ARARIA
BIHAR,PASHCHIM CHAMPARAN,SARAN,SARAN
BIHAR,PURBI CHAMPARAN,SARAN,SARAN
BIHAR,PURNIA,PURNEA,PURNEA
BIHAR,SAHARSA,SARAN,SARAN
BIHAR,SAMASTIPUR,BHAGALPUR,BHAGALPUR
BIHAR,SHEIKHPURA,PURNEA,PURNEA
BIHAR,SHEOHAR,SARAN,SARAN
BIHAR,SITAMARHI,SARAN,SARAN
BIHAR,SIWAN,SARAN,SARAN
BIHAR,VAISHALI,SUPAUL,SUPAUL
CHANDIGARH,CHANDIGARH,,
CHHATTISGARH,BALOD,BASTAR,BASTAR
CHHATTISGARH,BALODA BAZAR,BASTAR,BASTAR
CHHATTISGARH,BALRAMPUR,BILASPUR,BILASPUR
CHHATTISGARH,BEMETARA,BASTAR,BASTAR
CHHATTISGARH,BIJAPUR,BILASPUR,BILASPUR
CHHATTISGARH,DANTEWADA,BASTAR,BASTAR
CHHATTISGARH,DHAMTARI,BASTAR,BASTAR
CHHATTISGARH,GARIYABAND,SARGUJA,SARGUJA
CHHATTISGARH,GAURELLA-PENDRA-MARWAHI,BILASPUR,BILASPUR
CHHATTISGARH,JANJGIR-CHAMPA,JANJGIR,JANJGIR
CHHATTISGARH,JASHPUR,BILASPUR,BILASPUR
CHHATTISGARH,KABIRDHAM,SARGUJA,SARGUJA
CHHATTISGARH,KANKER,JANJGIR,JANJGIR
CHHATTISGARH,KONDAGAON,SARGUJA,SARGUJA
CHHATTISGARH,KORBA,RAIPUR,RAIPUR
CHHATTISGARH,KOREA,RAIPUR,RAIPUR
CHHATTISGARH,MAHASAMUND,BASTAR,BASTAR
CHHATTISGARH,MUNGELI,JANJGIR,JANJGIR
CHHATTISGARH,NARAYANPUR,RAIPUR,RAIPUR
CHHATTISGARH,RAIGARH,RAIPUR,RAIPUR
CHHATTISGARH,RAJNANDGAON,JANJGIR,JANJGIR
CHHATTISGARH,SUKMA,SARGUJA,SARGUJA
CHHATTISGARH,SURAJPUR,RAIPUR,RAIPUR
CHHATTISGARH,SURGUJA,SARGUJA,SARGUJA
DELHI,DELHI_TOTAL,SOUTH DELHI,SOUTH DELHI
GOA,NORTH GOA,SOUTH GOA,SOUTH GOA
GUJARAT,AHMADABAD,AHMEDABAD,AHMEDABAD
GUJARAT,ANAND,JUNAGAD,JUNAGAD
GUJARAT,ARAVALLI,AMRELI,AMRELI
GUJARAT,BANAS KANTHA,BANASKANTHA,BANASKANTHA
GUJARAT,BHARUCH,KUTCH,KUTCH
GUJARAT,BOTAD,AHMEDABAD,AHMEDABAD
GUJARAT,CHHOTAUDEPUR,VADODRA,VADODRA
GUJARAT,DANG,JAMNAGAR,JAMNAGAR
GUJARAT,DEVBHUMI DWARKA,SABARKANTHA,SABARKANTHA
GUJARAT,DOHAD,AHMEDABAD,AHMEDABAD
GUJARAT,GIR SOMNATH,SURAT,SURAT
GUJARAT,KACHCHH,KUTCH,KUTCH
GUJARAT,KHEDA,KAIRA KHEDA,KAIRA KHEDA
GUJARAT,MAHESANA,AHMEDABAD,AHMEDABAD
GUJARAT,MAHISAGAR,GANDHINAGAR,GANDHINAGAR
GUJARAT,MORBI,AMRELI,AMRELI
GUJARAT,NARMADA,JUNAGAD,JUNAGAD
GUJARAT,NAVSARI,VALSAR,VALSAR
GUJARAT,PANCH MAHALS,GANDHINAGAR,GANDHINAGAR
GUJARAT,PATAN,SURAT,SURAT
GUJARAT,PORBANDAR,BHAVNAGAR,BHAVNAGAR
GUJARAT,SABAR KANTHA,SABARKANTHA,SABARKANTHA
GUJARAT,TAPI,AMRELI,AMRELI
GUJARAT,VADODARA,VADODRA,VADODRA
GUJARAT,VALSAD,VALSAR,VALSAR
HARYANA,CHARKI DADRI,CHANDIGARH,CHANDIGARH
HARYANA,FARIDABAD,KARNAL,KARNAL
HARYANA,FATEHABAD,AMBALA,AMBALA
HARYANA,GURGAON,ROHTAK,ROHTAK
HARYANA,HISAR,HISSAR,HISSAR
HARYANA,JHAJJAR,CHANDIGARH,CHANDIGARH
HARYANA,JIND,BHIWANI,BHIWANI
HARYANA,KAITHAL,KARNAL,KARNAL
HARYANA,KURUKSHETRA,ROHTAK,ROHTAK
HARYANA,MAHENDRAGARH,CHANDIGARH,CHANDIGARH
HARYANA,MEWAT,BHIWANI,BHIWANI
HARYANA,PALWAL,AMBALA,AMBALA
HARYANA,PANCHKULA,AMBALA,AMBALA
HARYANA,PANIPAT,CHANDIGARH,CHANDIGARH
HARYANA,REWARI,BHIWANI,BHIWANI
HARYANA,SIRSA,HISSAR,HISSAR
HARYANA,SONIPAT,ROHTAK,ROHTAK
HARYANA,YAMUNANAGAR,CHANDIGARH,CHANDIGARH
HIMACHAL PRADESH,BILASPUR,KINNAUR,KINNAUR
HIMACHAL PRADESH,HAMIRPUR,KINNAUR,KINNAUR
HIMACHAL PRADESH,SIRMAUR,SHIMLA,SHIMLA
JAMMU AND KASHMIR,KISHTWAR,KUPWARA,KUPWARA
JAMMU AND KASHMIR,SAMBA,RAMBAN,RAMBAN
JHARKHAND,CHATRA,RANCHI,RANCHI
JHARKHAND,EAST SINGHBUM,EAST SINGHBHUM,EAST SINGHBHUM
JHARKHAND,GARHWA,RANCHI,RANCHI
JHARKHAND,GODDA,RANCHI,RANCHI
JHARKHAND,GUMLA,PALAMAU,PALAMAU
JHARKHAND,HAZARIBAGH,RANCHI,RANCHI
JHARKHAND,KODERMA,RANCHI,RANCHI
JHARKHAND,LATEHAR,PALAMAU,PALAMAU
JHARKHAND,LOHARDAGA,PALAMAU,PALAMAU
JHARKHAND,PALAMU,PALAMAU,PALAMAU
JHARKHAND,SAHEBGANJ,RANCHI,RANCHI
JHARKHAND,SARAIKELA KHARSAWAN,PALAMAU,PALAMAU
KARNATAKA,BANGALORE RURAL,BENGALURU,BENGALURU
KARNATAKA,CHAMARAJANAGAR,CHAMARAJNAGAR,CHAMARAJNAGAR
KARNATAKA,CHIKBALLAPUR,CHIKMAGALUR,CHIKMAGALUR
KARNATAKA,DAKSHIN KANNAD,DAKSHIN KANNADA,DAKSHIN KANNADA
KARNATAKA,DAVANGERE,DAVANAGERE,DAVANAGERE
KARNATAKA,RAMANAGARA,CHAMARAJNAGAR,CHAMARAJNAGAR
KARNATAKA,UDUPI,TUMKUR,TUMKUR
KARNATAKA,UTTAR KANNAD,UTTAR KANNADA,UTTAR KANNADA
KARNATAKA,YADGIR,GADAG,GADAG
KERALA,IDUKKI,THRISSUR,THRISSUR
KERALA,KASARAGOD,MALAPPURAM,MALAPPURAM
KERALA,PATHANAMTHITTA,THIRUVANATHA PURAM,THIRUVANATHA PURAM
KERALA,THIRUVANANTHAPURAM,THIRUVANATHA PURAM,THIRUVANATHA PURAM
KERALA,WAYANAD,PALAKKAD,PALAKKAD
MADHYA PRADESH,AGAR MALWA,SAGAR,SAGAR
MADHYA PRADESH,ANUPPUR,SHAJAPUR,SHAJAPUR
MADHYA PRADESH,ASHOKNAGAR,SAGAR,SAGAR
MADHYA PRADESH,BARWANI,REWA,REWA
MADHYA PRADESH,BHIND,INDORE,INDORE
MADHYA PRADESH,BURHANPUR,SHAJAPUR,SHAJAPUR
MADHYA PRADESH,DEWAS,REWA,REWA
MADHYA PRADESH,DINDORI,INDORE,INDORE
MADHYA PRADESH,HARDA,DHAR,DHAR
MADHYA PRADESH,JABALPUR,JABALPUR(A),JABALPUR(A)
MADHYA PRADESH,JHABUA,JABALPUR(A),JABALPUR(A)
MADHYA PRADESH,KATNI,SATNA,SATNA
MADHYA PRADESH,KHANDWA,MANDLA,MANDLA
MADHYA PRADESH,KHARGONE,DHAR,DHAR
MADHYA PRADESH,MANDSAUR,MANDLA,MANDLA
MADHYA PRADESH,MORENA,REWA,REWA
MADHYA PRADESH,NEEMUCH,DAMOH,DAMOH
MADHYA PRADESH,NIWARI,WEST NIMAR,WEST NIMAR
MADHYA PRADESH,PANNA,SATNA,SATNA
MADHYA PRADESH,RATLAM,SATNA,SATNA
MADHYA PRADESH,SEHORE,SEONI,SEONI
MADHYA PRADESH,SHAHDOL,MANDLA,MANDLA
MADHYA PRADESH,SINGRAULI,NARSINGHPUR,NARSINGHPUR
MADHYA PRADESH,VIDISHA,DHAR,DHAR
MAHARASHTRA,AHILYANAGAR,AHMEDNAGAR,AHMEDNAGAR
MAHARASHTRA,BEED,NADED,NADED
MAHARASHTRA,BHANDARA,CHANDRAPUR,CHANDRAPUR
MAHARASHTRA,BULDHANA,BULDANA,BULDANA
MAHARASHTRA,CHHATRAPATI SAMBHAJINAGAR,RATNAGIRI,RATNAGIRI
MAHARASHTRA,DHARASHIV,WASHIM,WASHIM
MAHARASHTRA,DHULE,PUNE,PUNE
MAHARASHTRA,GADCHIROLI,RATNAGIRI,RATNAGIRI
MAHARASHTRA,HINGOLI,SANGLI,SANGLI
MAHARASHTRA,JALNA,JALGAON,JALGAON
MAHARASHTRA,NANDED,NADED,NADED
MAHARASHTRA,NANDURBAR,NAGPUR,NAGPUR
MANIPUR,BISHNUPUR,IMPHAL EAST,IMPHAL EAST
MANIPUR,CHANDEL,IMPHAL EAST,IMPHAL EAST
MANIPUR,CHURACHANDPUR,IMPHAL EAST,IMPHAL EAST
MANIPUR,IMPHAL WEST,IMPHAL EAST,IMPHAL EAST
MANIPUR,SENAPATI,IMPHAL EAST,IMPHAL EAST
MANIPUR,TAMENGLONG,IMPHAL EAST,IMPHAL EAST
MANIPUR,THOUBAL,IMPHAL EAST,IMPHAL EAST
MANIPUR,UKHRUL,IMPHAL EAST,IMPHAL EAST
MEGHALAYA,EAST GARO HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,EAST JAINTIA HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,NORTH GARO HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,SOUTH GARO HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,SOUTH WEST GARO HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,SOUTH WEST KHASI HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,WEST GARO HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,WEST JAINTIA HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MEGHALAYA,WEST KHASI HILLS,EAST KHASI HILLS,EAST KHASI HILLS
MIZORAM,AIZAWL,AIZWAL,AIZWAL
MIZORAM,CHAMPHAI,AIZWAL,AIZWAL
MIZORAM,HNAHTHIAL,AIZWAL,AIZWAL
MIZORAM,KHAWZAWL,AIZWAL,AIZWAL
MIZORAM,KOLASIB,AIZWAL,AIZWAL
MIZORAM,LAWNGTLAI,AIZWAL,AIZWAL
MIZORAM,LUNGLEI,AIZWAL,AIZWAL
MIZORAM,MAMIT,AIZWAL,AIZWAL
MIZORAM,SAIHA,AIZWAL,AIZWAL
MIZORAM,SAITUAL,AIZWAL,AIZWAL
MIZORAM,SERCHHIP,AIZWAL,AIZWAL
PUDUCHERRY,KARAIKAL,PUDUCHERRY,PUDUCHERRY
PUDUCHERRY,MAHE,PUDUCHERRY,PUDUCHERRY
PUDUCHERRY,PONDICHERRY,PUDUCHERRY,PUDUCHERRY
PUDUCHERRY,YANAM,PUDUCHERRY,PUDUCHERRY
PUNJAB,BARNALA,PATIALA,PATIALA
PUNJAB,BATHINDA,PATIALA,PATIALA
PUNJAB,FATEHGARH SAHIB,AMRITSAR,AMRITSAR
PUNJAB,FAZILKA,PATIALA,PATIALA
PUNJAB,FIROZEPUR,HOSHIARPUR,HOSHIARPUR
PUNJAB,GURDASPUR,HOSHIARPUR,HOSHIARPUR
PUNJAB,JALANDHAR,LUDHIANA,LUDHIANA
PUNJAB,KAPURTHALA,PATIALA,PATIALA
PUNJAB,MANSA,LUDHIANA,LUDHIANA
PUNJAB,MUKTSAR,AMRITSAR,AMRITSAR
PUNJAB,NAWANSHAHR,HOSHIARPUR,HOSHIARPUR
PUNJAB,PATHANKOT,PATIALA,PATIALA
PUNJAB,RUPNAGAR,PATIALA,PATIALA
PUNJAB,S.A.S NAGAR,AMRITSAR,AMRITSAR
PUNJAB,SANGRUR,HOSHIARPUR,HOSHIARPUR
PUNJAB,TARN TARAN,PATIALA,PATIALA
RAJASTHAN,ALWAR,JHALAWAR,JHALAWAR
RAJASTHAN,BARAN,BANSWARA,BANSWARA
RAJASTHAN,BHARATPUR,JAIPUR,JAIPUR
RAJASTHAN,BHILWARA,JHALAWAR,JHALAWAR
RAJASTHAN,DAUSA,UDAIPUR,UDAIPUR
RAJASTHAN,DUNGARPUR,UDAIPUR,UDAIPUR
RAJASTHAN,GANGANAGAR,SRIGANGANAGAR,SRIGANGANAGAR
RAJASTHAN,HANUMANGARH,JHALAWAR,JHALAWAR
RAJASTHAN,JHUNJHUNU,CHURU,CHURU
RAJASTHAN,KARAULI,PALI,PALI
RAJASTHAN,NAGAUR,SRIGANGANAGAR,SRIGANGANAGAR
RAJASTHAN,PRATAPGARH,SRIGANGANAGAR,SRIGANGANAGAR
RAJASTHAN,RAJSAMAND,JAISALMER,JAISALMER
RAJASTHAN,SIKAR,BIKANER,BIKANER
TAMIL NADU,ARIYALUR,KARUR,KARUR
TAMIL NADU,CHENGALPATTU,NAGAPATTINAM,NAGAPATTINAM
TAMIL NADU,ERODE,VELLORE,VELLORE
TAMIL NADU,KALLAKURICHI,KANYAKUMARI,KANYAKUMARI
TAMIL NADU,KANCHIPURAM,RAMANATHAPURAM,RAMANATHAPURAM
TAMIL NADU,KANNIYAKUMARI,KANYAKUMARI,KANYAKUMARI
TAMIL NADU,KRISHNAGIRI,NILGIRIS,NILGIRIS
TAMIL NADU,MAYILADUTHURAI,MADURAI,MADURAI
TAMIL NADU,NAMAKKAL,PERAMBALUR,PERAMBALUR
TAMIL NADU,PUDUKKOTTAI,MADURAI,MADURAI
TAMIL NADU,RANIPET,MADURAI,MADURAI
TAMIL NADU,SIVAGANGA,VIRUDHUNAGAR,VIRUDHUNAGAR
TAMIL NADU,THE NILGIRIS,NILGIRIS,NILGIRIS
TAMIL NADU,THENI,CHENNAI,CHENNAI
TAMIL NADU,THENKASI,CHENNAI,CHENNAI
TAMIL NADU,THIRUVALLUR,TIRUVALLUR,TIRUVALLUR
TAMIL NADU,THIRUVARUR,TIRUVALLUR,TIRUVALLUR
TAMIL NADU,TIRUCHIRAPPALLI,TIRUCHIRAPALLI,TIRUCHIRAPALLI
TAMIL NADU,TIRUPATHUR,TIRUPATTUR,TIRUPATTUR
TAMIL NADU,TIRUPPUR,TIRUPATTUR,TIRUPATTUR
TAMIL NADU,TIRUVANNAMALAI,TIRUVALLUR,TIRUVALLUR
TAMIL NADU,VILLUPURAM,TIRUVALLUR,TIRUVALLUR
TELANGANA,KARIMNAGAR,MAHBOOB NAGAR,MAHBOOB NAGAR
TELANGANA,MAHBUBNAGAR,MAHBOOB NAGAR,MAHBOOB NAGAR
TELANGANA,RANGAREDDI,WARANGAL URBAN,WARANGAL URBAN
TELANGANA,WARANGAL,WARANGAL URBAN,WARANGAL URBAN
TRIPURA,DHALAI,NORTH TRIPURA,NORTH TRIPURA
TRIPURA,GOMATI,NORTH TRIPURA,NORTH TRIPURA
TRIPURA,KHOWAI,WEST TRIPURA,WEST TRIPURA
TRIPURA,SEPAHIJALA,WEST TRIPURA,WEST TRIPURA
TRIPURA,SOUTH TRIPURA,NORTH TRIPURA,NORTH TRIPURA
TRIPURA,UNAKOTI,NORTH TRIPURA,NORTH TRIPURA
UTTARAKHAND,ALMORA,HARIDWAR,HARIDWAR
UTTARAKHAND,BAGESHWAR,HARIDWAR,HARIDWAR
UTTARAKHAND,CHAMOLI,HARIDWAR,HARIDWAR
UTTARAKHAND,CHAMPAWAT,HARIDWAR,HARIDWAR
UTTARAKHAND,DEHRADUN,DEHRA DUN,DEHRA DUN
UTTARAKHAND,PAURI GARHWAL,NAINITAL,NAINITAL
UTTARAKHAND,PITHORAGARH,UDHAM SINGH NAGAR,UDHAM SINGH NAGAR
UTTARAKHAND,RUDRA PRAYAG,UDHAM SINGH NAGAR,UDHAM SINGH NAGAR
UTTARAKHAND,TEHRI GARHWAL,TEHRI NEW,TEHRI NEW
UTTARAKHAND,UDAM SINGH NAGAR,UDHAM SINGH NAGAR,UDHAM SINGH NAGAR
UTTARAKHAND,UTTAR KASHI,HARIDWAR,HARIDWAR
WEST BENGAL,24 PARAGANAS NORTH,SOUTH 24 PARGANAS,SOUTH 24 PARGANAS
WEST BENGAL,24 PARAGANAS SOUTH,SOUTH 24 PARGANAS,SOUTH 24 PARGANAS
WEST BENGAL,ALIPURDUAR,PURULIA,PURULIA
WEST BENGAL,COOCHBEHAR,COOCH BEHAR,COOCH BEHAR
WEST BENGAL,DINAJPUR DAKSHIN,SOUTH DINAJPUR,SOUTH DINAJPUR
WEST BENGAL,DINAJPUR UTTAR,SOUTH DINAJPUR,SOUTH DINAJPUR
WEST BENGAL,HOOGHLY,HOOGLY,HOOGLY
WEST BENGAL,JHARGRAM,HOWRAH,HOWRAH
WEST BENGAL,MEDINIPUR EAST,PASCHIM MEDNAPUR,PASCHIM MEDNAPUR
WEST BENGAL,MEDINIPUR WEST,PASCHIM MEDNAPUR,PASCHIM MEDNAPUR
"""

# Read into DataFrame
df_map = pd.read_csv(io.StringIO(csv_content))
print(f"Loaded {len(df_map)} manual mapping rules.")

# --- 3. APPLY CORRECTIONS ---
print("\nLoading Crop Data...")
df_crop = pd.read_excel(crop_file)

# Build Correction Dictionary: { 'VISAKHAPATANAM': 'VISAKHAPATNAM' }
# We filter out rows where correct name is missing
df_map = df_map.dropna(subset=['CORRECT_NAME_FROM_DB'])
correction_dict = dict(zip(df_map['Crop_District_Original'], df_map['CORRECT_NAME_FROM_DB']))

# Standardize Crop Data keys
df_crop['State_Upper'] = df_crop['State'].str.upper().str.strip()
df_crop['District_Upper'] = df_crop['District'].str.upper().str.strip()

# Create 'District_Final' column
# Logic: If in dict, use dict value. Else, use original upper case name.
df_crop['District_Final'] = df_crop['District_Upper'].map(correction_dict).fillna(df_crop['District_Upper'])

# --- 4. PREPARE RAINFALL DATA ---
print("Loading Rainfall Data...")
df_rain = pd.read_excel(rain_file)

# Aggregate Monthly -> Annual Rainfall
# Note: We group by State/District/Year.
rain_summary = df_rain.groupby(['State', 'District', 'Year'])['Rainfall (mm)'].sum().reset_index()
rain_summary.rename(columns={'Rainfall (mm)': 'Annual_Rainfall'}, inplace=True)

# Standardize Rainfall keys
rain_summary['State_Upper'] = rain_summary['State'].str.upper().str.strip()
rain_summary['District_Upper'] = rain_summary['District'].str.upper().str.strip()

# --- 5. MERGE ---
print("\nMerging Datasets...")

# Left Join: Keep all Crop records, attach Rain where available
df_merged = pd.merge(
    df_crop,
    rain_summary,
    how='left',
    left_on=['State_Upper', 'District_Final', 'Year'],
    right_on=['State_Upper', 'District_Upper', 'Year']
)

# --- 6. CLEAN UP & SAVE ---
# Select only useful columns
cols_to_keep = [
    'State_x', 'District_Final', 'Year', 'Crop',
    'Area', 'Production', 'Yield', 'Annual_Rainfall'
]

df_final = df_merged[cols_to_keep].copy()
df_final.rename(columns={
    'State_x': 'State',
    'District_Final': 'District'
}, inplace=True)

# Stats
total_rows = len(df_final)
matched_rows = df_final['Annual_Rainfall'].notna().sum()
match_percentage = (matched_rows / total_rows) * 100

print("-" * 30)
print(f"Total Crop Records: {total_rows}")
print(f"Records with Rainfall Data: {matched_rows}")
print(f"Final Match Rate: {match_percentage:.1f}%")
print("-" * 30)

# Save
df_final.to_excel(output_file, index=False)
print(f"\nSUCCESS! Master dataset saved to:\n{output_file}")

Loaded 333 manual mapping rules.

Loading Crop Data...
Loading Rainfall Data...

Merging Datasets...
------------------------------
Total Crop Records: 29578
Records with Rainfall Data: 20298
Final Match Rate: 68.6%
------------------------------

SUCCESS! Master dataset saved to:
rain_fall/results/Final_Merged_Dataset_Clean.xlsx


# Step 10: Final Engineering (Aggregation & Validation)

**Objective:**
Perform final cleanup logic to prepare the dataset for modeling and database storage.

**Processing Steps:**

### 1. Duplicate Handling
Aggregate duplicate seasonal entries (e.g., Kharif and Rabi) by summing:

- `Area`
- `Production`

### 2. Case-Sensitivity Fix
Ensure perfect joins by creating uppercase matching keys for `State`.

### 3. Missing Data Removal
Drop rows with `NaN` rainfall values to create an ML-ready dataset.

### Mathematical Logic

Final Yield is computed as:

$Yield_{Final} = \frac{\sum Production}{\sum Area}$

**Output:**
`Final_Engineered_Dataset.csv` (12,426 rows)

In [10]:
import pandas as pd
import os

# Paths
folder_path = r"rain_fall/results"
input_file = os.path.join(folder_path, "Final_Merged_Dataset_Clean.xlsx")
output_file = os.path.join(folder_path, "Final_Engineered_Dataset.csv")

print("--- STEP 5: FINAL ENGINEERING ---")

# 1. Load the Merged Data from Step 9
df = pd.read_excel(input_file)

# 2. AGGREGATE DUPLICATES (Seasonal Data)
print("Aggregating Seasonal Data...")
# We group by State, District, Year, and Crop
# We SUM Area and Production to get Annual totals
df_agg = df.groupby(['State', 'District', 'Year', 'Crop'], as_index=False)[['Area', 'Production']].sum()

# Recalculate Yield based on Annual Totals
# Avoid Division by Zero
df_agg['Yield'] = df_agg.apply(lambda row: row['Production'] / row['Area'] if row['Area'] > 0 else 0, axis=1)

# 3. RE-MERGE RAINFALL (To ensure no data loss during grouping)
# We need to grab the rainfall column again because the groupby might have dropped it if it wasn't numeric
# Or we can just take the first value since Rainfall is constant for that Year/District
rainfall_lookup = df[['State', 'District', 'Year', 'Annual_Rainfall']].drop_duplicates()
df_final = pd.merge(df_agg, rainfall_lookup, on=['State', 'District', 'Year'], how='left')

# 4. DROP MISSING RAINFALL
print("Filtering valid training data...")
rows_before = len(df_final)
df_final_clean = df_final.dropna(subset=['Annual_Rainfall'])
rows_after = len(df_final_clean)

# 5. SAVE
df_final_clean.to_csv(output_file, index=False)

print("-" * 30)
print(f"Original Aggregated Rows: {rows_before}")
print(f"Final ML-Ready Rows:      {rows_after}")
print("-" * 30)
print(f"SUCCESS! Final ML Dataset saved to:\n{output_file}")

--- STEP 5: FINAL ENGINEERING ---
Aggregating Seasonal Data...
Filtering valid training data...
------------------------------
Original Aggregated Rows: 17674
Final ML-Ready Rows:      12426
------------------------------
SUCCESS! Final ML Dataset saved to:
rain_fall/results/Final_Engineered_Dataset.csv


# Part 5: Final Database Architecture & Analysis

# Step 11: Building the Final OLTP Database (Normalization)

**Objective:**
Store the clean, merged dataset into a relational SQLite database using 3rd Normal Form (3NF) to minimize redundancy.

For example, long strings such as `"Andaman and Nicobar Islands"` are stored once and referenced using IDs.

---

## Schema Design

### Dimension Tables

- **States**
  - `StateID`
  - `StateName`

- **Districts**
  - `DistrictID`
  - `DistrictName`
  - `StateID` (Foreign Key)

- **Crops**
  - `CropID`
  - `CropName`

### Fact Table

- **Crop_Yield_Facts**
  - `FactID`
  - `Year`
  - `Area`
  - `Production`
  - `Yield`
  - `Rainfall`
  - `DistrictID` (Foreign Key)
  - `CropID` (Foreign Key)

**Output:**
`Final_Agri_Weather_OLTP.db`

In [11]:
import pandas as pd
import sqlite3
import os

# Paths
folder_path = r"rain_fall/results"
csv_file = os.path.join(folder_path, "Final_Engineered_Dataset.csv")
db_path = os.path.join(folder_path, "Final_Agri_Weather_OLTP.db")

print("--- STEP 11: BUILDING FINAL OLTP DATABASE ---")

# 1. Load the Master Dataset
df = pd.read_csv(csv_file)

# 2. Connect to Database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 3. Create Dimension Tables (Lookup Tables)

# --- A. STATES TABLE ---
# Get unique states
states = df['State'].unique()
df_states = pd.DataFrame(states, columns=['StateName']).sort_values('StateName').reset_index(drop=True)
df_states.reset_index(inplace=True)
df_states.rename(columns={'index': 'StateID'}, inplace=True)
df_states.to_sql('States', conn, if_exists='replace', index=False)

# --- B. DISTRICTS TABLE ---
# Get unique districts linked to states
dist_map = df[['State', 'District']].drop_duplicates().sort_values(['State', 'District'])
# Merge to get StateID
dist_map = dist_map.merge(df_states, left_on='State', right_on='StateName')
df_districts = dist_map[['District', 'StateID']].reset_index(drop=True)
df_districts.reset_index(inplace=True)
df_districts.rename(columns={'index': 'DistrictID', 'District': 'DistrictName'}, inplace=True)
df_districts.to_sql('Districts', conn, if_exists='replace', index=False)

# --- C. CROPS TABLE ---
# Get unique crops
crops = df['Crop'].unique()
df_crops = pd.DataFrame(crops, columns=['CropName']).sort_values('CropName').reset_index(drop=True)
df_crops.reset_index(inplace=True)
df_crops.rename(columns={'index': 'CropID'}, inplace=True)
df_crops.to_sql('Crops', conn, if_exists='replace', index=False)

# 4. Create Fact Table (Transactional Data)
# Replace names with IDs
fact_df = df.merge(df_crops, left_on='Crop', right_on='CropName')
fact_df = fact_df.merge(df_districts, left_on='District', right_on='DistrictName')

# Select columns for the final table
final_fact = fact_df[['DistrictID', 'CropID', 'Year', 'Area', 'Production', 'Yield', 'Annual_Rainfall']]
final_fact.to_sql('Crop_Yield_Facts', conn, if_exists='replace', index=False)

print("Database Created Successfully!")
print("-" * 30)
print(f"States Stored:    {len(df_states)}")
print(f"Districts Stored: {len(df_districts)}")
print(f"Crops Stored:     {len(df_crops)}")
print(f"Facts Stored:     {len(final_fact)}")
print("-" * 30)
print(f"Saved to: {db_path}")

conn.close()

--- STEP 11: BUILDING FINAL OLTP DATABASE ---
Database Created Successfully!
------------------------------
States Stored:    28
Districts Stored: 246
Crops Stored:     53
Facts Stored:     12426
------------------------------
Saved to: rain_fall/results/Final_Agri_Weather_OLTP.db


# Step 12: Final OLAP Analysis (Business Intelligence)

**Objective:**
Perform multi-dimensional analysis on the engineered dataset to extract meaningful insights.

We simulate an OLAP Cube using Pandas.

---

## Operations Performed

### 1. Roll-Up
Aggregate production by `State` to identify the highest producing regions.

### 2. Dice
Filter for a specific sub-cube, for example:
- Crop = Rice
- High rainfall years

### 3. Slice
Isolate a specific year (e.g., 2014) to compare crop performance.

### 4. Pivot
Create a cross-tabulation of Yield trends over the years.

### 5. Correlation Analysis
Analyze whether rainfall has a measurable impact on yield.

Example question:
Does increased rainfall significantly increase crop productivity?

---

**Final Outcome:**
A fully engineered Agriculture + Climate dataset stored in a normalized SQL database and analyzed using OLAP-style multi-dimensional operations.

In [12]:
import pandas as pd
import numpy as np

# Load the final data
file_path = r"rain_fall/results/Final_Engineered_Dataset.csv"
df = pd.read_csv(file_path)

print("--- STEP 12: OLAP BUSINESS INTELLIGENCE ---")

# --- 1. ROLL-UP (Aggregation) ---
# Question: Which State has the highest Total Agricultural Production?
print("\n[A] ROLL-UP: Total Production by State (Top 5)")
state_prod = df.groupby('State')['Production'].sum().sort_values(ascending=False).head(5)
print(state_prod)

# --- 2. DICE (Filter multiple dimensions) ---
# Question: How does Rice perform in years with Heavy Rainfall (>2000mm)?
print("\n[B] DICE: Rice Yield in High Rainfall Zones (>2000mm)")
high_rain_rice = df[
    (df['Crop'] == 'Rice') &
    (df['Annual_Rainfall'] > 2000)
    ]
avg_yield_high = high_rain_rice['Yield'].mean()
avg_yield_all = df[df['Crop'] == 'Rice']['Yield'].mean()

print(f"Average Rice Yield (All Conditions): {avg_yield_all:.2f} Tonnes/Ha")
print(f"Average Rice Yield (High Rain):      {avg_yield_high:.2f} Tonnes/Ha")
if avg_yield_high > avg_yield_all:
    print(">> Insight: Rice thrives in high rainfall areas.")
else:
    print(">> Insight: Excessive rain might be damaging rice crops.")

# --- 3. SLICE (Filter one dimension) ---
# Question: What were the top crops in 2014?
print("\n[C] SLICE: Top 3 Crops by Area in 2014")
slice_2014 = df[df['Year'] == 2014].groupby('Crop')['Area'].sum().sort_values(ascending=False).head(3)
print(slice_2014)

# --- 4. PIVOT (Cross-Tabulation) ---
# Question: How has the Yield of 'Coconut' changed over the years in different states?
print("\n[D] PIVOT: Coconut Yield Trend (Select States)")
coconut_df = df[(df['Crop'] == 'Coconut') & (df['State'].isin(['Kerala', 'Tamil Nadu', 'Karnataka']))]
pivot = pd.pivot_table(coconut_df, values='Yield', index='Year', columns='State', aggfunc='mean')
print(pivot.tail(5)) # Show last 5 years

# --- 5. CORRELATION INSIGHT ---
# Question: What is the correlation between Rain and Yield across the whole dataset?
corr = df['Annual_Rainfall'].corr(df['Yield'])
print(f"\n[E] GLOBAL CORRELATION: Rainfall vs Yield = {corr:.4f}")
print("Note: A low number is expected because different crops have different water needs.")

--- STEP 12: OLAP BUSINESS INTELLIGENCE ---

[A] ROLL-UP: Total Production by State (Top 5)
State
Kerala            5.932168e+10
Karnataka         3.972231e+10
Tamil Nadu        3.632468e+10
Andhra Pradesh    1.649057e+10
West Bengal       2.747263e+09
Name: Production, dtype: float64

[B] DICE: Rice Yield in High Rainfall Zones (>2000mm)
Average Rice Yield (All Conditions): nan Tonnes/Ha
Average Rice Yield (High Rain):      nan Tonnes/Ha
>> Insight: Excessive rain might be damaging rice crops.

[C] SLICE: Top 3 Crops by Area in 2014
Crop
Guar seed    4801934.00
Sugarcane    1859561.06
Coconut      1642284.00
Name: Area, dtype: float64

[D] PIVOT: Coconut Yield Trend (Select States)
State    Karnataka       Kerala    Tamil Nadu
Year                                         
2016   7989.577877  6351.872046  12357.866089
2017   8235.625545  6401.232011  11239.796569
2018   7311.646460  6414.736487  12377.821626
2019   8325.006031  5894.120740   9410.338391
2020      8.113498  5807.458044 

In [13]:
df.head()

,State,District,Year,Crop,Area,Production,Yield,Annual_Rainfall
0,Andaman and Nicobar Islands,NICOBAR,2012,Banana,241.0,2034.0,8.439834,5815.4
1,Andaman and Nicobar Islands,NICOBAR,2012,Coconut,14650.0,89800000.0,6129.692833,5815.4
2,Andaman and Nicobar Islands,NICOBAR,2012,Tapioca,61.0,523.0,8.573770,5815.4
3,Andaman and Nicobar Islands,NICOBAR,2013,Banana,170.0,1300.5,7.650000,6011.0
4,Andaman and Nicobar Islands,NICOBAR,2013,Coconut,14655.0,96200000.0,6564.312521,6011.0


In [14]:

# Create SQLite connection
conn = sqlite3.connect(":memory:")   

# Load dataframe into SQLite
df.to_sql("final_engineered_dataset", conn, index=False, if_exists="replace")

print("Table created successfully!")

Table created successfully!


In [15]:
df.columns


Index(['State', 'District', 'Year', 'Crop', 'Area', 'Production', 'Yield',
       'Annual_Rainfall'],
      dtype='str')

Year-wise Total Rainfall

In [16]:
query1 = """
SELECT
    Year,
    SUM(Annual_Rainfall) AS Total_Annual_Rainfall
FROM final_engineered_dataset
GROUP BY Year
ORDER BY Year;
"""

pd.read_sql_query(query1, conn)


,Year,Total_Annual_Rainfall
0,2010,2339602.5
1,2011,2506611.6
2,2012,2168750.9
3,2013,2741874.9
4,2014,2005768.5
5,2015,1952498.4
6,2016,1499573.8
7,2017,1664085.1
8,2018,1683295.7
9,2019,1728442.5


State-wise Rainfall Analysis

In [17]:
query2 = """
SELECT
    State,
    SUM(Annual_Rainfall) AS Total_Rainfall
FROM final_engineered_dataset
GROUP BY State
ORDER BY Total_Rainfall DESC;
"""

pd.read_sql_query(query2, conn)


,State,Total_Rainfall
0,Karnataka,4997372.4
1,Kerala,4616685.2
2,Tamil Nadu,2517827.1
3,Assam,2389000.6
4,Madhya Pradesh,1418759.5
5,Meghalaya,799337.9
6,Gujarat,745752.9
7,Andhra Pradesh,687557.0
8,Chhattisgarh,601913.2
9,Rajasthan,532038.4


Crop-wise Average Rainfall

In [18]:
query3 = """
SELECT
    Crop,
    ROUND(AVG(Annual_Rainfall), 2) AS Avg_Annual_Rainfall
FROM final_engineered_dataset
GROUP BY Crop
ORDER BY Avg_Annual_Rainfall DESC;
"""

pd.read_sql_query(query3, conn)


,Crop,Avg_Annual_Rainfall
0,Oilseeds total,3839.71
1,Maize,3610.15
2,Cowpea(Lobia),3140.38
3,Drum Stick,2925.95
4,Cardamom,2873.72
5,Mango,2788.87
6,Wheat,2590.38
7,Groundnut,2501.65
8,Black pepper,2474.00
9,Ragi,2453.67


Year-wise Crop Production Analysis

In [19]:
query4 = """
SELECT
    Year,
    Crop,
    SUM(Production) AS Total_Production
FROM final_engineered_dataset
GROUP BY Year, Crop
ORDER BY Year, Total_Production DESC;
"""

pd.read_sql_query(query4, conn)


,Year,Crop,Total_Production
0,2010,Coconut,1.065286e+10
1,2010,Sugarcane,1.493744e+08
2,2010,Banana,5.869002e+06
3,2010,Potato,3.189057e+06
4,2010,Onion,3.109546e+06
...,...,...,...
361,2020,Maize,1.169600e+02
362,2020,Sunflower,1.830000e+01
363,2020,Wheat,3.000000e+00
364,2020,Rapeseed &Mustard,2.000000e+00


Top 5 Districts by Average Rainfall

In [20]:
query5 = """
SELECT
    District,
    State,
    ROUND(AVG(Annual_Rainfall), 2) AS Avg_Rainfall
FROM final_engineered_dataset
GROUP BY District
ORDER BY Avg_Rainfall DESC
LIMIT 5;
"""

pd.read_sql_query(query5, conn)


,District,State,Avg_Rainfall
0,EAST KHASI HILLS,Meghalaya,12222.70
1,UTTAR KANNADA,Karnataka,10206.01
2,DAKSHIN KANNADA,Karnataka,7270.12
3,SHIMOGA,Karnataka,6687.63
4,ERNAKULAM,Kerala,5772.54
